In [3]:
import re
import numpy as np

# we are given data in a file format, we need to open the file, read its contents. Each item is separated by some
# symbol, generally it is a linebreak sometimes it is two. Then we want to parse the actual text into some format of
# data that we can work with, this means we will have to write our own parsing function for each level and pass it
# into this function
def get_data(day: int, parser=str, sep="\n") -> list:
    with open(f"./inputs/p{day}.txt", "r") as f:
        contents = f.read().strip().split(sep)
    return [parser(content) for content in contents]

### Day 1
https://adventofcode.com/2021/day/1

In [13]:
arr_1 = get_data(1, parser=int)

In [14]:
# Part 1
n_inc = 0
for i in range(1, len(arr_1)):
    if arr_1[i-1] < arr_1[i]:
        n_inc += 1
n_inc

1602

In [15]:
# Part 2
n_inc = 0
for i in range(2, len(arr_1)):
    if arr_1[i-1] < arr_1[i]:
        n_inc += 1
n_inc

1601

### Day 2

In [16]:
def parser(content):
    k, v = content.split(" ")
    return [k, int(v)]
        
parser("forward 5")

['forward', 5]

In [17]:
arr_2 = get_data(2, parser=parser)

In [19]:
# Part 1
horz = depth = 0
for e in arr_2:
    if e[0] == "forward":
        horz += e[1]
    elif e[0] == "down":
        depth += e[1]
    else:
        depth -= e[1]
print(horz, depth, horz * depth)

1980 951 1882980


In [20]:
# Part 2
horz = depth = aim = 0
for e in arr_2:
    if e[0] == "forward":
        horz += e[1]
        depth += aim * e[1]
    elif e[0] == "down":
        aim += e[1]
    else:
        aim -= e[1]
print(horz, depth, horz * depth)

1980 995572 1971232560


### Day 3

In [25]:
arr_3 = get_data(3, parser=str)
arr_3[0]

'100000101101'

In [26]:
# Part 1
sol = []
for c in range(len(arr_3[0])):
    bit = 0
    for r in range(len(arr_3)):
        bit += int(arr_3[r][c]) # iterate over the columns
    sol.append(round(bit/len(arr_3) * 2 // 1))

power = len(sol) - 1
gam = eps = 0
for bit in sol:
    gam += bit * 2 ** power
    eps += (1 - bit) * 2 ** power
    power -= 1
print(gam, eps, gam * eps)

1337 2758 3687446


In [56]:
# 011000111111 1599
# 101011000100 2756
# 1599 * 2756 = 4406844

# Part 2 sorting - it's like doing a binary sort across two indices
arr_ox = np.sort(np.array(arr_3))
arr_co = np.copy(arr_ox)

for col in range(len(arr_ox[0])):
    if len(arr_ox) == 1:
        break
    if len(arr_ox) % 2 == 1:
        ox_bit = int(arr_ox[len(arr_ox) // 2][col])
        ind_ox = [True if int(x[col]) == ox_bit else False for x in arr_ox]
    else:
        ox_bit = max(int(arr_ox[len(arr_ox) // 2][col]), int(arr_ox[len(arr_ox) // 2 - 1][col]))
        ind_ox = [True if int(x[col]) == ox_bit else False for x in arr_ox]
    arr_ox = arr_ox[ind_ox]
    
print(arr_ox)

for col in range(len(arr_co[0])):
    if len(arr_co) == 1:
        break
    if len(arr_co) % 2 == 1:
        co_bit = 1 - int(arr_co[len(arr_co) // 2][col])
        ind_co = [True if int(x[col]) == co_bit else False for x in arr_co]
    else:
        co_bit = min(1 - int(arr_co[len(arr_co) // 2][col]), 1 - int(arr_co[len(arr_co) // 2 - 1][col]))
        ind_co = [True if int(x[col]) == co_bit else False for x in arr_co]
    arr_co = arr_co[ind_co]
    
print(arr_co)

['011000111111']
['101011000100']


### Day 4

In [214]:
class Board:
    def __init__(self, nums: list[list[int]]):
        self.nums = nums
        self.blen = len(nums[0])
        
    def is_win(self):
        for i in range(self.blen):
            if len(set(self.nums[i, :])) == 1 and self.nums[i][0] == -1:
                return True
            if len(set(self.nums[:, i])) == 1 and self.nums[0][i] == -1:
                return True
        return False
    
    def replace_val(self, val):
        r, c = np.where(self.nums == val)
        self.nums[r[0], c[0]] = -1
    
    def contains_val(self, val):
        if val in self.nums:
            return True
        return False
    
    def calc_score(self, num):
        tot = 0
        for row in range(self.blen): 
            for col in range(self.blen): 
                if self.nums[row][col] != -1:
                    tot += self.nums[row][col]
        return num * tot
        
    def __str__(self):
        s = ""
        for i in range(self.blen):
            s += f"{self.nums[i]} \n"
        s += "\n"
        return s

    def __repr__(self):
        return str(self)

        
# pop the first line and \n\n out of the file
def get_data_4():
    def parser(content):
        content = content.split("\n")
        for i in range(len(content)):
            content[i] = [int(e) for e in content[i].split(" ") if e != '']
        return np.array(content)
        
    with open("./inputs/p4.txt", "r") as f:
        nums = [int(e) for e in f.readline().strip().split(",")]
        f.readline() # skip the line so that we begin content with the first board
        contents = f.read().strip().split("\n\n")
                
    return nums, [Board(parser(content)) for content in contents]

In [239]:
# Part 1
len_board = 5
def get_winner():
    nums, boards = get_data_4()
    for num in nums:
        for board in boards:
            if board.contains_val(num):
                board.replace_val(num)
                if board.is_win():
                    return board.calc_score(num)
    return -1

sol = get_winner()
sol

39984

In [277]:
# Part 2
def get_loser():
    nums, boards = get_data_4()
    for num in nums:
        for i in range(len(boards)-1, -1, -1):
            if boards[i].contains_val(num):
                boards[i].replace_val(num)
                if boards[i].is_win():
                    if len(boards) == 1:
                        return boards[0].calc_score(num)
                    del boards[i]
    return -1

sol = get_loser()
sol

# takeaway... use numpy arrays... they are so useful for column comparisons!

8468

### Day 5

In [33]:
def parser(content):
    regex = ",| -> |\+" # matches exactly a comma OR a space arrow space
    return [int(coord) for coord in re.split(regex, content)]
    
print(parser("409,872 -> 409,963"))
arr = get_data(5, parser=parser)

[409, 872, 409, 963]


In [111]:
# Part 1
dim = 1000
sol = np.zeros((dim, dim), dtype=np.int64)

for line in arr:
    x1, x2 = min(line[0], line[2]), max(line[0], line[2])
    y1, y2 = min(line[1], line[3]), max(line[1], line[3])
        
    # vertical line
    if x1 == x2:
        for y in range(y1, y2+1):
            sol[x1,y] += 1
    
    # horizontal line
    elif y1 == y2:
        for x in range(x1, x2+1):
            sol[x,y1] += 1

np.count_nonzero(sol > 1)

7436

In [118]:
# Part 2
dim = 1000
sol = np.zeros((dim, dim), dtype=np.int64)

for line in arr:
    x1, x2 = line[0], line[2]
    y1, y2 = line[1], line[3]
        
    # vertical line
    if x1 == x2:
        y1, y2 = min(line[1], line[3]), max(line[1], line[3])
        for y in range(y1, y2+1):
            sol[x1, y] += 1
    
    # horizontal line
    elif y1 == y2:
        x1, x2 = min(line[0], line[2]), max(line[0], line[2])
        for x in range(x1, x2+1):
            sol[x, y1] += 1
            
    # diagonal line
    else:
        x_slope = 1 if x1 < x2 else -1
        y_slope = 1 if y1 < y2 else -1
        r = max(x1, x2) - min(x1, x2)
        for i in range(r+1):
            x_ind = x1 + i * x_slope
            y_ind = y1 + i * y_slope
            sol[x_ind, y_ind] += 1
                
np.count_nonzero(sol > 1)

# I was rather careless initially not checking if the range was increasing or decreasing and then in the second part
# not checking if the line was increasing or decreasing, I was way too frusturated on a rather simple accumulation problem

21104

### Day 6

In [26]:
with open(f"./inputs/p6.txt", "r") as f:
    arr = [int(i) for i in f.read().strip().split(",")]

In [6]:
# Part 1 => d=80
days = 80
for _ in range(days):
    # we want to keep this range fixed if we are appending to the list
    for i in range(len(arr)):
        arr[i] -= 1
        if arr[i] < 0:
            arr[i] = 6
            arr.append(8)
len(arr)
# reset arr...

394994

In [24]:
# old fish 6 days to spawn new fish, new fish 8 days
samp = [1,3,5,1]
ds = 50

"""
f(d, n) where n is time until spawn, and d is days left
we can create a lookup table of n=8 and d=80

days=> 8, 6, 6, 6, 6

f(64, 4) => f(60, 8) + f(60, 6) + 1

f(d, n) => f(d-n, 8) + f(d-n, 6) + 1
We get a new fish (8 days) and the current fish resets (6) and we add 1 to the total
f(d, n) = 0 if d <= n

if d > n:
    compute new fish
else:
    return

How can we prevent ourselves from computing f(60, 8) twice... there will be another starting fish that has to compute this same value.

We can work backwards and fill the table? Iterate through D and N, put value for number of fish that are created.

Select d,n for each element in the input and sum them together!
"""
acc = []
def calc_fish(d, n, acc):
    if d > n:
        acc.append(1)
        # print(d, n, acc)
        calc_fish(d-n, 8, acc) + calc_fish(d-n, 6, acc)
    return acc

In [27]:
##########
# REVIEW #
##########

# Part 2
days = [0] * 9
# Update the current numbers
for fish in arr:
    days[fish] += 1
for i in range(256):
    # To make it cyclic: 0, 1, 2, 3, 4, 5, 6, 7, 8    
    today = i % len(days)
    # Add new babies
    days[(today + 7) % len(days)] += days[today]
print(f'Total lanternfish after 256 days: {sum(days)}')

Total lanternfish after 256 days: 1765974267455
